## 데이터 및 라이브러리 로딩

In [70]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [71]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


In [72]:
import pandas as pd
import seaborn as sns
import os, sys, shutil
import matplotlib.pyplot as plt
from pycaret.regression import *
from sklearn.ensemble import RandomForestRegressor


In [73]:
ROOT_DIR = './'
DATA_ROOT_DIR = os.path.join(ROOT_DIR, 'Data')

In [74]:
train = pd.read_csv(os.path.join(DATA_ROOT_DIR, 'movies_train.csv'))
test = pd.read_csv(os.path.join(DATA_ROOT_DIR, 'movies_test.csv'))
submission = pd.read_csv(os.path.join(DATA_ROOT_DIR, 'submission.csv'))

## 데이터탐색

In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [76]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [77]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1


## 데이터 전처리

In [78]:
# Replace nan with zero
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [79]:
# 감독명 : 너무 다양해서 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

In [80]:
# 제목 : 의미가 없기 때문에 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

In [81]:
train.distributor.value_counts()

CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: distributor, Length: 169, dtype: int64

In [82]:
# 상위 5개의 배급사를 제외하고 '기타'로처리
distributor_list = train.distributor.value_counts()[:5]
def func(distributor):
    if distributor in distributor_list:
        return distributor
    else:
        return '기타'

train['distributor'] = train['distributor'].apply(lambda x : func(x))
test['distributor'] = test['distributor'].apply(lambda x : func(x))


In [83]:
train.drop(['box_off_num'], axis=1)

,distributor,genre,release_time,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,0.00,0,91,2
1,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,1161602.50,2,387,3
2,(주)쇼박스,액션,2013-06-05,123,15세 관람가,220775.25,4,343,4
3,(주)NEW,코미디,2012-07-12,101,전체 관람가,23894.00,2,20,6
4,기타,코미디,2010-11-04,108,15세 관람가,1.00,1,251,2
...,...,...,...,...,...,...,...,...,...
595,(주)NEW,드라마,2014-08-13,111,청소년 관람불가,3833.00,1,510,7
596,(주)쇼박스,드라마,2013-03-14,127,15세 관람가,496061.00,1,286,6
597,(주)마운틴픽쳐스,공포,2010-09-30,99,청소년 관람불가,0.00,0,123,4
598,CJ 엔터테인먼트,느와르,2015-05-14,102,15세 관람가,0.00,0,431,4


In [93]:
numeric_col = ['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']
categori_col = ['distributor', 'genre', 'screening_rat']

In [107]:
s = setup(train, target='box_off_num', numeric_features=numeric_col, ignore_features=categori_col)

,Description,Value
0,session_id,4809
1,Target,box_off_num
2,Original Data,"(600, 10)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(419, 24)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True,
                                      features_todrop=['distributor', 'genre',
                                                       'screening_rat'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['time',
                                                          'dir_prev_bfnum',
                                                          'dir_prev_num',
                                                          'num_staff',
                                                          'num_actor'],
                                      target='box_off_num', time_features=[])),
                ('imputer',
                 Sim

In [108]:
top3 = compare_models(n_select=3, fold=5, sort="RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,7.819712e+05,2.362648e+12,1.466474e+06,0.2675,4.0375,1934.0809,0.046
omp,Orthogonal Matching Pursuit,7.694069e+05,2.363361e+12,1.474102e+06,0.2505,3.9207,1144.8854,0.020
lr,Linear Regression,8.413478e+05,2.440944e+12,1.505112e+06,0.2140,4.2371,1761.6198,0.028
lightgbm,Light Gradient Boosting Machine,7.537538e+05,2.427069e+12,1.507739e+06,0.1859,3.6868,451.3162,0.040
ridge,Ridge Regression,8.415707e+05,2.458764e+12,1.511413e+06,0.2059,4.2381,1695.2493,0.036
lasso,Lasso Regression,8.460371e+05,2.497022e+12,1.526671e+06,0.1842,4.2325,1666.7658,0.026
llar,Lasso Least Angle Regression,8.483061e+05,2.499445e+12,1.527313e+06,0.1837,4.2339,1666.4888,0.022
lar,Least Angle Regression,8.527384e+05,2.506606e+12,1.529887e+06,0.1805,4.2439,1718.0559,0.042
rf,Random Forest Regressor,6.859462e+05,2.517612e+12,1.530683e+06,0.1800,3.0319,724.1320,0.616
huber,Huber Regressor,6.190766e+05,2.968014e+12,1.653102e+06,0.0478,5.6000,2787.9090,0.036


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:[ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=4809, selection='cyclic', tol=0.0001, warm_start=False), OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=None,
                          normalize=True, precompute='auto', tol=None), LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)]
INFO:logs:compare_models() succesfully completed......................................


In [109]:
blender_top3 = blend_models(top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,6.435180e+05,1.037531e+12,1.018593e+06,0.3619,4.0553,295.4815
1,7.416035e+05,1.236179e+12,1.111836e+06,0.2820,3.6612,216.2069
2,6.193200e+05,9.413199e+11,9.702164e+05,-0.0992,4.3799,7494.5472
3,7.168859e+05,1.889900e+12,1.374736e+06,0.2989,3.4719,121.5554
4,4.998267e+05,5.755740e+11,7.586659e+05,-1.0141,3.8553,291.6604
5,7.466421e+05,1.684695e+12,1.297958e+06,0.3178,3.9716,1230.6348
6,9.400397e+05,5.519856e+12,2.349437e+06,0.3067,3.5628,109.8148
7,8.357459e+05,3.448485e+12,1.857010e+06,0.2041,4.5675,7769.9282
8,1.000298e+06,3.657346e+12,1.912419e+06,0.3217,4.4820,573.3951


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('en',
                             ElasticNet(alpha=1.0, copy_X=True,
                                        fit_intercept=True, l1_ratio=0.5,
                                        max_iter=1000, normalize=False,
                                        positive=False, precompute=False,
                                        random_state=4809, selection='cyclic',
                                        tol=0.0001, warm_start=False)),
                            ('omp',
                             OrthogonalMatchingPursuit(fit_intercept=True,
                                                       n_nonzero_coefs=None,
                                                       normalize=True,
                                                       precompute='auto',
                                                       tol=None)),
               

In [110]:
pred = predict_model(blender_top3, data=test)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingRegressor(estimators=[('en',
                             ElasticNet(alpha=1.0, copy_X=True,
                                        fit_intercept=True, l1_ratio=0.5,
                                        max_iter=1000, normalize=False,
                                        positive=False, precompute=False,
                                        random_state=4809, selection='cyclic',
                                        tol=0.0001, warm_start=False)),
                            ('omp',
                             OrthogonalMatchingPursuit(fit_intercept=True,
                                                       n_nonzero_coefs=None,
                                                       normalize=True,
                                                       precompute='auto',
                                                       tol=None)),
                            ('lr',
                      

In [112]:
submission['box_off_num'] = pred.Label

In [113]:
submission.to_csv('AutoML.csv', index=False)